In [10]:
import data_loader
import model
import loss
import solver

import torch
from torch import optim, nn
from tqdm import tqdm
import pickle

In [11]:
root_path = '/home/renyan/ossdata/massachusetts-roads-dataset/'
road_path = root_path + "tiff_select2_parts_16/"
INPUT_SIZE, OUTPUT_SIZE = 256, 256
BATCH_SIZE = 4
LR = 0.0002
EPOCH_NUM = 20

In [12]:
train_dataset = data_loader.RoadDataset(road_path, INPUT_SIZE, OUTPUT_SIZE, 'train')
val_dataset = data_loader.RoadDataset(road_path, INPUT_SIZE, OUTPUT_SIZE, 'val')

train dataset set: 7056 pairs
val dataset set: 224 pairs


# LinkNet34

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = model.LinkNet34().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(params = net.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)

In [5]:
sv = solver.Solver(device, net, train_dataset, val_dataset, criterion, LR, BATCH_SIZE, optimizer, scheduler, "LinkNet34")

Net LinkNet34 initialized.


In [ ]:
sv.train(epochs = EPOCH_NUM, save_cp = True, dir_checkpoint = 'checkpoints/', prefix = 'linknet34_lr_test')

Epoch 1/20:   5%|▍         | 348/7056 [01:03<20:48,  5.37img/s, loss (batch)=0.693]

In [13]:
# with open('sv_linknet.pickle','wb') as p:
#     pickle.dump(sv, p)

# DLinkNet

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = model.DLinkNet34().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(params = net.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 50, gamma = 0.5)

In [14]:
sv = solver.Solver(device, net, train_dataset, val_dataset, criterion, LR, BATCH_SIZE, optimizer, scheduler, "DLinkNet34")

Net DLinkNet34 initialized.


In [ ]:
sv.train(epochs = EPOCH_NUM, save_cp = True, dir_checkpoint = 'checkpoints/', prefix = 'dlinknet34')

Epoch 3/20:  83%|████████▎ | 5888/7056 [21:36<03:01,  6.44img/s, loss (batch)=0.67]   